This notebook contains examples of prompt engineering and LLM API (OpenAI) usage. Inspired by [DAIR.AI](https://github.com/dair-ai) guides.

In [37]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [38]:
# API configuration
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [39]:
# set model parameters
def set_open_params(
    model="text-davinci-003",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):

    openai_params = {}
    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, prompt):

    response = openai.Completion.create(
        engine = params['model'],
        prompt = prompt,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

## Basic prompt: text completion

In [10]:
params = set_open_params()

prompt = "The human brain is"

response = get_completion(params, prompt)

In [11]:
IPython.display.Markdown(response.choices[0].text)

 the most complex organ in the human body. It is responsible for controlling many of the body’s functions, processing sensory information, and creating thoughts and memories. The brain contains billions of neurons, which are connected to each other by synapses. These neurons communicate with each other through electrical and chemical signals. The brain is divided into four main sections, which are the cerebrum, the cerebellum, the brainstem, and the hypothalamus. Each of these sections has different functions and is responsible for different aspects of the brain’s activities.

In [12]:
# changing temperature
params = set_open_params(temperature=0)
prompt = "The human brain is"
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 the most complex organ in the human body. It is responsible for controlling and coordinating body functions, as well as processing information from the senses. It is made up of billions of neurons that communicate with each other through electrical and chemical signals. The brain is divided into four main parts: the cerebrum, the cerebellum, the brainstem, and the hypothalamus. Each part has its own unique functions and roles in the body. The cerebrum is the largest part of the brain and is responsible for higher-level thinking, such as problem solving, decision making, and language. The cerebellum is responsible for coordinating movement and balance. The brainstem is responsible for controlling basic body functions, such as breathing, heart rate, and blood pressure. The hypothalamus is responsible for regulating hormones and controlling hunger, thirst, and body temperature.

## Question Answering

In [13]:
params = set_open_params()
prompt = '''Question: What is a gravitational wave?
Answer:'''
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 A gravitational wave is a ripple in the fabric of spacetime caused by the acceleration of a large mass. These waves are predicted by Albert Einstein's general theory of relativity, and are detectable by measuring tiny changes in the distance between two distant objects.

In [14]:
params = set_open_params()
prompt = '''What is a gravitational wave?'''
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



A gravitational wave is a rippling in the fabric of space-time caused by the acceleration of a massive object. It is a type of wave that can travel through space and time, carrying with it information about its source and the nature of gravity that produced it.

## Context-based Question Answering

In [15]:
prompt = """Answer the question based on the context below. Keep the answer short and concise.
Respond "Not sure" if unable to get the answer.

Context: Esperanto is the world's most widely spoken constructed international auxiliary language.
Created by the ophthalmologist L. L. Zamenhof in 1887, it was intended to be a universal second language
for international communication, or "the international language". Zamenhof first described the language
in Dr. Esperanto's International Language, which he published under the pseudonym Doktoro Esperanto.
Early adopters of the language liked the name and soon used it to describe his language. 

Question: Where does the name Esperanto come from? 

Answer:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 The name Esperanto comes from Dr. Esperanto's International Language, a pseudonym used by L. L. Zamenhof when publishing the language.

## Text classification

In [17]:
prompt = """Classify the text into neutral, negative or positive.

Text: This movie was not so bad.

Sentiment:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Neutral

In [18]:
prompt = """Classify the text into neutral, negative or positive.

Text: This movie was so bad that it's ridiculously good.

Sentiment:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Positive

In [19]:
# now for something trickier: sarcasm detection
prompt = """Classify the text's tone into sarcastic or neutral.

Text: My coffee machine broke again. That's just perfect.

Tone: Sarcastic

Text: I didn't watch the movie yesterday. Maybe next time.

Tone: Neutral

Text: They finally got their package delivered.

Tone: Neutral

Text: He's going to fire the whole department, how lovely.

Tone: Sarcastic

Text: Great, the cat is scratching the new sofa.

Tone:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Sarcastic

## Code generation

In [20]:
prompt = 'Write a Python function that solves Hanoi Tower puzzle'

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



def hanoi_tower(disk_number, source, auxiliary, destination):
    if disk_number == 1:
        print("Move disk 1 from", source, "to", destination)
        return
    
    hanoi_tower(disk_number-1, source, destination, auxiliary)
    print("Move disk", disk_number, "from", source, "to", destination)
    hanoi_tower(disk_number-1, auxiliary, source, destination)

hanoi_tower(3, "A", "B", "C")

In [21]:
prompt = '''Table users, columns = [UserId, Age]
Table items, columns = [ItemId, Price]
Table purchases, columns = [PurchaseId, ItemId, UserId, PurchaseDate]
Create a query to get average purchase sum per month by 18-24 year old users'''

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



SELECT 
    YEAR(purchases.PurchaseDate) AS Year, 
    MONTH(purchases.PurchaseDate) AS Month, 
    AVG(items.Price) AS AvgPurchaseSum 
FROM purchases 
JOIN items 
    ON purchases.ItemId = items.ItemId 
JOIN users 
    ON purchases.UserId = users.UserId 
WHERE users.Age BETWEEN 18 AND 24 
GROUP BY Year, Month;

## Role playing: language learning practice

In [41]:
prompt = """The following is a speaking practice in German with a teacher who also speaks English.
The teacher is friendly, corrects mistakes and provides explanation in English.

Teacher: Guten Morgen! Heute sprechen wir über Haustiere. Hast du ein Haustier? 
Student: Ja, ich habe ein Katze.
Teacher: 'Katze' is a feminine noun, so it goes with a feminine article 'eine'. 
Student: Okay, ich habe eine Katze.
Teacher: Super! Wie heißt sie?
Student: Sie heißt Mia.
Teacher: Und wie alt ist Mia?
Student: Sie ist vier Jahren alt.
Teacher:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 'Jahren' is the plural form of 'Jahr', so you need to say 'Sie ist vier Jahre alt.'

## Chain-of-thought prompting

In [24]:
prompt = """Question: Find the probability of getting two heads when five coins are tossed.
Solution: Number of ways of getting two heads = C(5,2) = 10. Total number of ways = 2**5 = 32.
P(two heads) = 10/32 = 5/16. The answer is 5/16.

Question: Find the probability of getting three tails when four coins are tossed.
Solution:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Number of ways of getting three tails = C(4,3) = 4. Total number of ways = 2**4 = 16. 
P(three tails) = 4/16 = 1/4. The answer is 1/4.

## Zero-shot Chain-of-thought

In [25]:
prompt = """Three friends of mine are coming over for dinner. I'd like to make pasta for everyone including myself.
One package contains 250 grams of spaghetti. An average serving size is about 100 grams.
How many packages should I buy?

Let's think step by step."""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



First, calculate how many servings are needed: 4 people x 1 serving each = 4 servings.

Second, calculate how many grams of spaghetti are needed: 4 servings x 100 grams per serving = 400 grams.

Finally, calculate how many packages are needed: 400 grams of spaghetti divided by 250 grams per package = 1.6 packages.

Therefore, you should buy 2 packages of spaghetti.

## Using Langchain

In [26]:
# web search
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [29]:
llm = OpenAI(temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [33]:
agent.run("When did the Berlin Wall fall? How many days passed since it came down as of today?")



> Entering new AgentExecutor chain...
 I need to find out when the Berlin Wall fell and then calculate the number of days since then.
Action: Search
Action Input: "When did the Berlin Wall fall?"
Observation: November 9, 1989
Thought: I need to calculate the number of days since then
Action: Calculator
Action Input: Today's date minus November 9, 1989
Observation: Answer: 12198

Thought: I now know the final answer
Final Answer: The Berlin Wall fell on November 9, 1989 and it has been 12198 days since then.

> Finished chain.


'The Berlin Wall fell on November 9, 1989 and it has been 12198 days since then.'

In [34]:
# prompt templates
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [42]:
chat = ChatOpenAI(temperature=0)

In [44]:
USER_INPUT = "I recently moved to Hamburg."
FINAL_PROMPT = """Translate the text into German. 

Text: {user_input}. 
Translation:"""

chat([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='Ich bin vor kurzem nach Hamburg gezogen.', additional_kwargs={})

In [54]:
template = '''You are an assistant that can classify the topic of input texts.
The labels you can use are {topic_labels}. Name the topic of the following sentence:'''
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [58]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(topic_labels="news, sports, science",
                               user_input='''On Wednesday, the king was received with military honors on Pariser Platz
                               in front of the Brandenburg Gate.''')
     .to_messages())

AIMessage(content='The topic of the following sentence is news.', additional_kwargs={})

In [57]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(topic_labels="news, sports, science",
                               user_input='''Functional neurologic disorder (FND), also known as conversion disorder
                               and functional neurologic symptom disorder, refers to a group of common neurological
                               movement disorders caused by an abnormality in how the brain functions.''')
     .to_messages())

AIMessage(content='The topic of the following sentence is science.', additional_kwargs={})